In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
from fpipe.pipeline.pipeline import run_pipeline
from fpipe.timestream import data_base as db 
from fpipe.timestream import bandpass_cal
from fpipe.timestream import destripe
from fpipe.timestream import rm_baseline
from fpipe.fnoise import fnoise

from fpipe.plot import plot_bandpass
from fpipe.plot import plot_waterfall
from fpipe.plot import plot_cal

import numpy as np
import os

from collections import OrderedDict as dict
from fpipe.point_source import abs_cal
from astropy.coordinates.angles import Angle

data_base = '/home/ycli/data/fanalysis/'
band_list = ['_1050-1150MHz', '_1150-1250MHz', '_1250-1450MHz']
tnoise_file= '/home/ycli/data/fast/tnoise/Tnoise_M_low.h5'
fwhm_file = '/home/ycli/code/fpipe/fpipe/data/fwhm.dat'
beam_list = [[16, 17, 18], [6, 7, 15, 19], [1, 2, 5, 8, 14], [3, 4, 9, 13], [10, 11, 12]]

In [ ]:
@run_pipeline
def gt_cal():
    
    pipe_tasks = []
    pipe_outdir = data_base
    pipe_logging = 'info'
    pipe_copy = False
    pipe_feedback = 0
    pipe_timing = True
    
    feed_select = list(np.arange(1, 20))
    

    pipe_tasks.append((destripe.TimeVar_Cal, 'gtcal1_'))
    pipe_tasks.append((bandpass_cal.Apply_EtaA, 'etaA1_'))
    
    #gtcal1_input_files  = ['sir_flagging_sumfeeds/%s_1050-1150MHz.h5'%f for f in file_list]
    gtcal1_input_files  = [] #['rb4_sumfeeds_nocal/%s_1050-1150MHz.h5'%f for f in file_list]
    gtcal1_output_files = [] #['gtcal_sumfeeds/%s_1050-1150MHz.h5'%f for f in file_list]
    gtcal1_gt_file = '' #data_base + '/gtgnu/%s_NoInterp.h5'%suffix
    gtcal1_ps_file = '' #data_base + '/gtps/%s.h5'%(suffix)
    
    etaA1_input_files  = [] #gtcal1_output_files
    etaA1_output_files = [] #['etaA_sumfeeds/%s_1050-1150MHz.h5'%f for f in file_list]
    etaA1_eta_A = [] #eta_A

    return locals()

@run_pipeline
def rm_bl():
    
    pipe_tasks = []
    pipe_outdir = data_base
    pipe_logging = 'info'
    pipe_copy = False
    pipe_feedback = 0
    pipe_timing = True
    
    feed_select = list(np.arange(1, 20))
    
    pipe_tasks.append((rm_baseline.Remove_Baseline, 'rmbsl1_'))
    rmbsl1_input_files  = [] #etaA1_output_files
    rmbsl1_output_files = [] #['rmbsl_sumfeeds/%s_1050-1150MHz.h5'%f for f in file_list]
    rmbsl1_baseline_file = '' #data_base + '/baseline/%s_%s.h5'%(DATA, DATE)

    return locals()


In [ ]:
DATA_Key_list = ['20210313', '20210314', '20210305', '20210302', '20210306', '20210307', '20210309', 
                 #'20220210', '20220211', '20220212', '20220213', '20220214', '20220222',
                 #'20220215', '20220216', '20220217', '20220218', '20220219', '20220220',
                ]

#DATA_Key_list = ['20210313',]

def iter_data_key(DATA_Key_list):
    
    for DATA_Key in DATA_Key_list:
    
        data = db.DATA_BASE[DATA_Key]
        DATA = data['DATA']
        DATE = data['DATE']
        cal_date = data['DATE_CAL']
        
        yield DATA_Key, DATA, DATE, cal_date

In [ ]:
%reset -f array

In [ ]:
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    file_root = data_base + '/rb4_sumfeeds/%s/'%DATE
    
    file_list = [
        ['%s/%s_arcdrift%04d-%04d_1050-1150MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
        ['%s/%s_arcdrift%04d-%04d_1150-1250MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
        ['%s/%s_arcdrift%04d-%04d_1250-1450MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
    ]
    
    output_name = data_base + '/gtgnu/%s_%s_NoInterp.h5'%(DATE, DATA)
    print(output_name)
    bandpass_cal.output_gtgnu(file_list, output_name, Tnoise_file=tnoise_file, interp_mask=False)
    %reset -f array
    
    gtps_file = data_base + '/gtps/%s_%s.h5'%(DATE, DATA)
    print(gtps_file)
    fnoise.est_gt_ps(file_list, tnoise_file, avg_len=None, output=gtps_file)
    %reset -f array

In [ ]:
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    file_root = data_base + '/rb4_sumfeeds/%s/'%DATE
    
    file_list = [
        ['%s/%s_arcdrift%04d-%04d_1050-1150MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
        ['%s/%s_arcdrift%04d-%04d_1150-1250MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
        ['%s/%s_arcdrift%04d-%04d_1250-1450MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
    ]
    
    gtps_file = data_base + '/gtps/%s_%s.h5'%(DATE, DATA)
    output_name = data_base + '/gtgnu/%s_%s_NoInterp.h5'%(DATE, DATA)
    
    title  = '%s \n %s'%(DATE, DATA.replace('_', ' '))
    fig_output = data_base + '/plot/gtps_%s_%s.png'%(DATE, DATA)
    print(fig_output)
    plot_bandpass.plot_gt_ps(gtps_file, title=title, output=fig_output, ymin=5.e-4, ymax=8.e-1)
    %reset -f array
    
    fig_output = data_base + '/plot/gtgnu_%s_%s_XX.png'%(DATE, DATA)
    print(fig_output)
    plot_bandpass.plot_gtgnu(output_name, title=DATA.replace('_', ' '), pol=0,
            norm=False, output=fig_output)
    %reset -f array
    
    fig_output = data_base + '/plot/gtgnu_%s_%s_YY.png'%(DATE, DATA)
    print(fig_output)
    plot_bandpass.plot_gtgnu(output_name, title=DATA.replace('_', ' '), pol=1,
            norm=False, output=fig_output)
    %reset -f array
    
    fig_output = data_base + '/plot/gt_%s_%s.png'%(DATE, DATA)
    print(fig_output)
    plot_bandpass.plot_gt(output_name, fk=0.05, title=DATA.replace('_', ' '),
            output=fig_output, gtps_file=gtps_file)
    %reset -f array

In [ ]:
#suffix = '%s_%s'%(DATE, DATA)
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    
    file_list = ['%s/%s_arcdrift%04d-%04d'%(DATE, DATA, i, i) for i in range(1, 8)]
    eta_A = data_base + '/eta/eta_%s.h5'%cal_date
    
    for band in band_list:
    
        gt_cal['gtcal1_input_files'] = ['rb4_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        gt_cal['gtcal1_output_files']= ['gtcal_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        gt_cal['gtcal1_gt_file'] = data_base + '/gtgnu/%s_%s_NoInterp.h5'%(DATE, DATA)
        gt_cal['gtcal1_ps_file'] = data_base + '/gtps/%s_%s.h5'%(DATE, DATA)
        
        gt_cal['etaA1_input_files'] = ['gtcal_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        gt_cal['etaA1_output_files']= ['etaA_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        gt_cal['etaA1_eta_A'] = eta_A
    
        gt_cal()
        %reset -f array

In [ ]:
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    
    file_root = data_base + '/etaA_sumfeeds/%s/'%DATE
    
    file_list = [
        #['%s/%s_arcdrift%04d-%04d_1050-1150MHz.h5'%(
        #    file_root, DATA, i, i) for i in range(1, 8)],
        #['%s/%s_arcdrift%04d-%04d_1150-1250MHz.h5'%(
        #    file_root, DATA, i, i) for i in range(1, 8)],
        ['%s/%s_arcdrift%04d-%04d_1250-1450MHz.h5'%(
            file_root, DATA, i, i) for i in range(1, 8)],
    ]
    
    output_name = data_base + '/baseline/%s_%s.h5'%(DATA, DATE)
    print(output_name)
    rm_baseline.output_baseline(file_list, output_name)
    
    fig_output_name = data_base + '/plot/baseline_%s_%s.png'%(DATA, DATE)
    plot_bandpass.plot_baseline(output_name, fig_output_name, utc=False)

In [ ]:
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    
    file_root = data_base + '/etaA_sumfeeds/%s/'%DATE
    for band in band_list[-1:]:
        file_list = [
            ['%s/%s_arcdrift%04d-%04d%s.h5'%(file_root, DATA, i, i, band) for i in range(1, 8)],
        ]
        
        fig_output_name = data_base + '/plot/baseline_%s_%s%s.png'%(DATA, DATE, band)
        plot_bandpass.plot_baseline(file_list, fig_output_name, utc=True)

In [ ]:
for DATA_Key, DATA, DATE, cal_date in iter_data_key(DATA_Key_list):
    
    file_list = ['%s/%s_arcdrift%04d-%04d'%(DATE, DATA, i, i) for i in range(1, 8)]
    eta_A = data_base + '/eta/eta_%s.h5'%cal_date
    
    for band in band_list:
    
        rm_bl['rmbsl1_input_files']   = ['etaA_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        rm_bl['rmbsl1_output_files']  = ['rmbsl_sumfeeds/%s%s.h5'%(f, band) for f in file_list]
        rm_bl['rmbsl1_baseline_file'] = data_base + '/baseline/%s_%s.h5'%(DATA, DATE)
    
        rm_bl()
        %reset -f array